# 章節 15：維度處理

## 15.6 折疊階層式結構

所有下層的分佈都是用相同的資料更新，所以前 m 個參數都是相同的。我們可以移除下層的 Dirichlet 分佈假設並且合併到上層。Species2 實作這個優化：

<pre>
class Species2(object):
    def __init__(self, ns):
        # ns 是多少物種的陣列
        self.ns = ns
        
        # 一序列對應的機率
        self.probs = numpy.ones(len(ns), dtype=numpy.double)
        
        # Dirichlet 的參數，初始化都為 1
        self.params = numpy.ones(self.high, dtype=numpy.int)
        
</pre>

Species2.Update 更新所有階層：先更新每個物種 n 的機率，再來更新 Dirichlet 參數：

<pre>
# class Species2
    def Update(self, data):
        like = numpy.zeros(len(self.ns), dtype=numpy.double)
        for i in range(1000):
            like += self.SampleLikelihood(data)
        self.probs *= like
        self.probs /= self.probs.sum()
        
        # 更新 Dirichlet 參數
        m = len(data)
        self.params[:m] += data
</pre>

- SampleLikelihood 方法回傳對每個 n 值似然性的陣列。並且將 1000 次取樣的似然性結果加總。
- self.probs 乘上加總的似然性並且正規化。
- 最後兩行是更新參數，就是 Dirichlet.Update 方法中做的事情

現在來看 SampleLikelihood 方法。有兩個地方可以優化：
1. 當假設的物種種類, n, 超過觀察到的總, m, 我們只需要前 m 個多項式 pmf 的參數；剩下的為 1。
1. 如果物種種類很多，資料的似然性可能會出現浮點下溢（floating-point underflow）。所以用 log 似然性計算會比較好（參考 10.5 小節）。

多項式 PMF 是
Again, the multinomial PMF is

$$ c_{x} p_{1}^{x_{1}} p_{2}^{x_{2}} ... p_{n}^{x_{n}} $$

所以 log 似然性為

$$ logc_{x}+x_{1}logp_{1}+...+x_{n}logp_{n} $$

這計算起來簡單且快速。再一次 $c_{x}$ 對每個假設來說都一樣（是個常數），所以我們可以拿掉它。這裡是程式碼：

<pre>
# class Species2
    def SampleLikelihood(self, data):
    
        # gammas 是一個序列，用 self.params 為參數從 gamma 分佈取樣的結果；序列長度為最多的物種種類 n 。
       r
        m = len(data)
        
        # row 是前 m 個 gammas 的參數；因為只有前 m 個參數跟資料有關。
        row = gammas[:m]
        
        # cumsum 方法會計算前 m 個累積的總和。e.g. np.cumsum([1, 1, 2]) => [1, 2, 4]
        col = numpy.cumsum(gammas)
        log_likes = []
        
        # 迭代所有的 n 並且將 log 似然性的總和存成一個序列。
        for n in self.ns:
            # 對每個 n 我們將 row 除以前 n 個 gamma 數值的總和。
            ps = row / col[n-1]
            
            terms = data * numpy.log(ps)
            
            # log_like: 包含 xi log pi 的總和（計算似然性）
            log_like = terms.sum()
            log_likes.append(log_like)
            
        # 將最大的 log 似然性平移到 0 避免最後線性的似然性太小(參考 10.5 小節) 
        log_likes -= numpy.max(log_likes)
        
        # 轉換成線性的似然性
        likes = numpy.exp(log_likes)
        coefs = [thinkbayes.BinomialCoef(n, m) for n in self.ns]
        likes *= coefs
        return likes
</pre>



Finally, before we return the likelihood, we have to apply a correction factor, which is the number of ways we could have observed these m species, if the total number of species is n. BinomialCoefficient computes “n choose m”, which is written (mn ).
As often happens, the optimized version is less readable and more error- prone than the original. But that’s one reason I think it is a good idea to start with the simple version; we can use it for regression testing. I plotted results from both versions and confirmed that they are approximately equal, and that they converge as the number of iterations increases.